In [1]:
# load packages
import pandas as pd
import numpy as np

from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# load train-test set
raw_train_test = pd.read_csv("../datasets/raw/applications_train_test_set.csv")

# display
raw_train_test.head(3)

,id,credit_event,name_contract_type,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,name_type_suite,name_income_type,name_education_type,name_housing_type,region_population_relative,days_employed,days_registration,days_id_publish,own_car_age,flag_mobil,flag_emp_phone,flag_work_phone,flag_cont_mobile,flag_phone,flag_email,occupation_type,cnt_fam_members,region_rating_client,region_rating_client_w_city,weekday_appr_process_start,hour_appr_process_start,reg_region_not_live_region,reg_region_not_work_region,live_region_not_work_region,reg_city_not_live_city,reg_city_not_work_city,live_city_not_work_city,organization_type,ext_source_1,ext_source_2,ext_source_3,apartments_avg,basementarea_avg,years_beginexpluatation_avg,years_build_avg,commonarea_avg,elevators_avg,entrances_avg,floorsmax_avg,floorsmin_avg,landarea_avg,livingapartments_avg,livingarea_avg,nonlivingapartments_avg,nonlivingarea_avg,apartments_mode,basementarea_mode,years_beginexpluatation_mode,years_build_mode,commonarea_mode,elevators_mode,entrances_mode,floorsmax_mode,floorsmin_mode,landarea_mode,livingapartments_mode,livingarea_mode,nonlivingapartments_mode,nonlivingarea_mode,apartments_medi,basementarea_medi,years_beginexpluatation_medi,years_build_medi,commonarea_medi,elevators_medi,entrances_medi,floorsmax_medi,floorsmin_medi,landarea_medi,livingapartments_medi,livingarea_medi,nonlivingapartments_medi,nonlivingarea_medi,fondkapremont_mode,housetype_mode,totalarea_mode,wallsmaterial_mode,emergencystate_mode,obs_30_cnt_social_circle,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_2,flag_document_3,flag_document_4,flag_document_5,flag_document_6,flag_document_7,flag_document_8,flag_document_9,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year,sk_id_prev_count,cnt_instalment_future_sum,sk_dpd_max,sk_dpd_sum,sk_dpd_def_max,sk_dpd_def_sum,sk_id_bureau_count,credit_day_overdue_sum,cnt_credit_prolong_avg,amt_credit_sum_sum,amt_credit_sum_debt_sum,amt_credit_sum_limit_sum,amt_credit_sum_overdue_sum,amt_annuity_sum,active_sum,amt_balance_avg_avg,amt_drawings_atm_current_avg_avg,amt_drawings_current_avg_avg,amt_drawings_other_current_avg_avg,amt_drawings_pos_current_avg_avg,amt_inst_min_regularity_avg_avg,amt_payment_current_avg_avg,amt_payment_total_current_avg_avg,amt_receivable_principal_avg_avg,amt_recivable_avg_avg,amt_total_receivable_avg_avg,cnt_drawings_atm_current_avg_avg,cnt_drawings_current_avg_avg,cnt_drawings_other_current_avg_avg,cnt_drawings_pos_current_avg_avg,cnt_instalment_mature_cum_avg_avg,sk_dpd_avg_avg,sk_dpd_def_avg_avg,negative_balance_avg_avg,negative_balance_sum_sum,count,installment_paiement_delay_min_min,installment_paiement_delay_max_max,installment_paiement_delay_avg_avg,amount_instalment_difference_min_min,amount_instalment_difference_max_max,amount_instalment_difference_avg_avg,prev_amt_annuity_sum,prev_amt_application_sum,prev_amt_credit_sum,prev_amt_down_payment_sum,prev_amt_goods_price_sum,prev_rate_down_payment_avg,prev_rate_interest_primary_avg,prev_rate_interest_privileged_avg,prev_days_decision_avg,prev_name_payment_type_count,prev_cnt_payment_sum,prev_days_first_drawing_avg,prev_days_first_due_avg,prev_days_last_due_1st_version_avg,prev_days_last_due_avg,prev_days_termination_avg,prev_nflag_insured_on_approval_avg
0,288213,0,Cash loans,False,False,0,225000.0,953460.0,62703.0,900000.0,Unaccompanied,Commercial associate,Secondary / secondary special,House / apartment,0.04622,116,4106.0,912,NaN,1,1,0,1,0,0,Realty agents,1.0,1,1,MONDAY,12,0,0,0,0,0,0,Services,0.702389,0.655313,NaN,0.0742,0.0503,0.9871,NaN,NaN,0.08,0.0690,

In [4]:
# drop id dan target column
features = raw_train_test.drop(['credit_event', 'id'], axis=1)

for feature in features:
    if raw_train_test[feature].dtype.kind in 'ifc':
        if len(raw_train_test[feature].value_counts()) > 6: 
            raw_train_test[feature] = pd.cut(raw_train_test[feature], 6)

In [8]:
Y = np.ravel(np.array(raw_train_test[['credit_event']]))
X = np.array(raw_train_test.drop(['credit_event', 'id'], axis=1))

print(Y, X)

[0 0 0 ... 0 0 0] [['Cash loans' False False ...
  Interval(-3234.109, 58485.5, closed='right')
  Interval(-3143.018, 58561.333, closed='right')
  Interval(-0.001, 0.167, closed='right')]
 ['Cash loans' False True ... Interval(58485.5, 119837.0, closed='right')
  Interval(58561.333, 119897.667, closed='right')
  Interval(0.333, 0.5, closed='right')]
 ['Cash loans' False True ... nan nan nan]
 ...
 ['Revolving loans' True False ...
  Interval(181188.5, 242540.0, closed='right')
  Interval(181234.0, 242570.333, closed='right')
  Interval(0.167, 0.333, closed='right')]
 ['Cash loans' False True ...
  Interval(181188.5, 242540.0, closed='right')
  Interval(181234.0, 242570.333, closed='right')
  Interval(0.333, 0.5, closed='right')]
 ['Cash loans' False False ... nan nan nan]]


In [ ]:
scores = pd.DataFrame()
oe = OrdinalEncoder()

for i, feature in enumerate(features):
    X_feature = X[:, i].astype(str).reshape(-1, 1)
    oe.fit(X_feature)
    X_enc = oe.transform(X_feature)
    iv = information_value(X=X_enc, Y=Y)
    chi = chi2(X=X_enc, y=Y)
    scores = scores.append(pd.DataFrame({'feature': [feature], 'information_value': [iv],
                                         'chi2_pvalue': [chi[1][0]]}))